In [2]:
import requests

def query_ollama(prompt):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "mistral", "prompt": prompt, "stream": False}
    )
    return response.json()['response']

# Test prompt
prompt = """
Why might Chennai Super Kings win the match?
- Win streak: 4
- Opponent's streak: 1
- Toss won: Yes
- Toss decision: Field
- Venue: Chennai
"""
print("🧠 LLM Reasoning:\n", query_ollama(prompt))


🧠 LLM Reasoning:
  There are several reasons why Chennai Super Kings (CSK) might win the match based on the provided data:

1. Win Streak: CSK has a current winning streak of 4 matches, while their opponent has only won once in their last game. This shows that CSK is currently playing well and consistent, which can give them an advantage in the upcoming match.
2. Toss Won: Winning the toss allows a team to decide whether to bat or field first. In this case, CSK won the toss and decided to field first, indicating that they might prefer chasing the total if they aren't confident about their batting lineup. However, the decision to field first also implies that they believe their bowling attack can restrict the opponent's scoring.
3. Venue: The match is being held in Chennai, a venue where CSK has traditionally performed well. The team's familiarity with the ground conditions and crowd support could give them an edge over their opponents.
4. Opponent's Streak: Although it's only one win, 

In [ ]:
 Evaluation of the  LLM Output (Ollama+Mistral)

In [ ]:
TASK-2
Backend FAST API 

In [3]:
!pip install fastapi uvicorn


     ---------------------------------------- 0.0/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/6


[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from fastapi import FastAPI
from pydantic import BaseModel
import requests

app = FastAPI()

class MatchInput(BaseModel):
    team1: str
    team2: str
    venue: str
    toss_winner: str
    toss_decision: str
    team1_streak: int
    team2_streak: int

def query_ollama(prompt):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "mistral", "prompt": prompt, "stream": False}
    )
    return response.json()['response']

def build_prompt(winner, features):
    return f"""
You are a cricket analyst. Explain in 3–5 sentences why the model predicted {winner} to win.

Factors:
- Team 1 streak: {features['team1_streak']}
- Team 2 streak: {features['team2_streak']}
- Toss won by: {features['toss_winner']}
- Toss decision: {features['toss_decision']}
- Venue: {features['venue']}
"""

@app.post("/predict-winner")
def predict(input: MatchInput):
    winner = input.team1 if input.team1_streak > input.team2_streak else input.team2

    prompt = build_prompt(winner, input.dict())
    reasoning = query_ollama(prompt)

    return {
        "predicted_winner": winner,
        "llm_reasoning": reasoning
    }
